In [7]:
#from u1_geopandas_v2.u1_streaming import incoming_dir
#!pip install -r Christine/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'Christine/requirements.txt'


In [1]:
!pip install matplotlib contextily

  Using cached contextily-1.6.2-py3-none-any.whl.metadata (2.9 kB)
  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached mercantile-1.2.1-py3-none-any.whl.metadata (4.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached affine-2.4.0-py3-none-any.whl.metadata (4.0 kB)
Using cached contextily-1.6.2-py3-none-any.whl (17 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 9.9 MB/s eta 0:00:00:00:0100:01
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Using cached affine-2.4.0-py3-none-any.whl (15 kB)


In [3]:
import os
import pandas as pd
import geopandas as gpd
from shapely import wkb
import time
from datetime import datetime

import folium
from folium import plugins
from shapely.geometry import Point, Polygon
import datetime

from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

In [18]:
def opprett_mappestruktur(data_mappe="./data"):
    """
    Oppretter nødvendig mappestruktur for konvertering.

    Args:
        data_mappe: Sti til hovedmappen for data

    Returns:
        dict: Stier til opprettede mapper
    """
    # Definer mappestruktur
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")
    source_mappe = os.path.join(data_mappe, "source")
    incoming_dir_mappe = os.path.join(data_mappe, "incoming")

    # Opprett mapper
    os.makedirs(data_mappe, exist_ok=True)
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)
    os.makedirs(source_mappe, exist_ok=True)
    os.makedirs(incoming_dir_mappe, exist_ok=True)

    print(f"Mappestruktur opprettet.")

    # Returner stier for senere bruk
    return {
        "data_mappe": os.path.abspath(data_mappe),
        "rå_mappe": os.path.abspath(rå_mappe),
        "prosessert_mappe": os.path.abspath(prosessert_mappe),
        "source_mappe": os.path.abspath(source_mappe),
        "incoming_mappe": os.path.abspath(incoming_dir_mappe)
    }

def legg_til_tidspartisjoneringskolonner(df, tid_kolonne='date_time_utc'):
    """
    Legger til kolonner for tidspartisjonering (kun time for partisjonering, men beholder dato-kolonner).
    """
    if tid_kolonne not in df.columns:
        print(f"Advarsel: Tidsstempelkolonne '{tid_kolonne}' finnes ikke")
        return df

    if not pd.api.types.is_datetime64_any_dtype(df[tid_kolonne]):
        print(f"Advarsel: Kolonnen '{tid_kolonne}' er ikke en datetime-kolonne")
        return df

    df_med_tid = df.copy()
    df_med_tid['year'] = df_med_tid[tid_kolonne].dt.year
    df_med_tid['month'] = df_med_tid[tid_kolonne].dt.month
    df_med_tid['day'] = df_med_tid[tid_kolonne].dt.day
    df_med_tid['hour'] = df_med_tid[tid_kolonne].dt.hour

    return df_med_tid

def opprett_geodataframe(df):
    """
    Oppretter en GeoDataFrame fra en DataFrame ved å finne koordinater eller geometrikolonner.
    """
    from shapely import wkt
    # Sjekk for lat/long kolonner
    if 'longitude' in df.columns and 'latitude' in df.columns:
        try:
            return gpd.GeoDataFrame(
                df,
                geometry=gpd.points_from_xy(df.longitude, df.latitude),
                crs="EPSG:4326"
            )
        except Exception as e:
            print(f"Kunne ikke opprette geometri fra lat/long: {e}")

    # Sjekk for andre geometrikolonner
    geom_kolonner = [col for col in df.columns if any(
        term in col.lower() for term in ['geom', 'coord', 'point', 'polygon', 'linestring', 'wkt']
    )]

    for col in geom_kolonner:
        if df[col].dtype != 'object':
            continue

        try:
            geom = df[col].apply(wkt.loads)
            return gpd.GeoDataFrame(df, geometry=geom, crs="EPSG:4326")
        except Exception:
            continue

    return None

def lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner):
    """
    Lagrer en GeoDataFrame som partisjonert GeoParquet.
    """
    from shapely import wkb # NB!! Ikke fjern!

    # Konverterer geometri til WKB for å kunne partisjonere
    df_med_wkb = gdf.copy()
    df_med_wkb['geometry_wkb'] = df_med_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    df_for_partisjon = df_med_wkb.drop(columns=['geometry'])

    # Utfør partisjonering
    df_for_partisjon.to_parquet(målfilsti, partition_cols=partisjon_kolonner)

    # Konverter hver partisjonert fil tilbake til GeoParquet
    konverter_partisjonerte_filer_til_geoparquet(målfilsti)
    return målfilsti

def opprett_år_mappe(år, målfilsti):
    """Oppretter en mappe for et spesifikt år."""
    år_mappe = os.path.join(målfilsti, f"year={år}")
    os.makedirs(år_mappe, exist_ok=True)
    return år_mappe

def forbered_wkb_dataframe(år_df):
    """Forbereder en dataframe med WKB-konvertert geometri for partisjonering."""
    # Konverter geometri til WKB for å kunne partisjonere
    år_df_wkb = år_df.copy()
    år_df_wkb['geometry_wkb'] = år_df_wkb['geometry'].apply(lambda geom: wkb.dumps(geom))
    år_df_wkb = år_df_wkb.drop(columns=['geometry'])
    return år_df_wkb

def konverter_fil_til_geoparquet(src_file, dst_file):
    """Konverterer en enkelt fil fra WKB-format til GeoParquet."""
    try:
        part_df = pd.read_parquet(src_file)
        if 'geometry_wkb' not in part_df.columns:
            return False

        part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
        part_df = part_df.drop(columns=['geometry_wkb'])
        part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

        # Sørg for at målmappen eksisterer
        os.makedirs(os.path.dirname(dst_file), exist_ok=True)
        part_gdf.to_parquet(dst_file)
        return True
    except Exception as e:
        print(f"Advarsel: Kunne ikke konvertere {src_file} til GeoParquet: {e}")
        return False

def kopier_og_konverter_filer(temp_år_mappe, år_mappe):
    """Kopierer og konverterer filer fra temp-mappen til målmappen."""
    suksess_count = 0
    feil_count = 0

    # Opprett først alle mappene
    for root, dirs, _ in os.walk(temp_år_mappe):
        for directory in dirs:
            src_dir = os.path.join(root, directory)
            rel_path = os.path.relpath(src_dir, temp_år_mappe)
            dst_dir = os.path.join(år_mappe, rel_path)
            os.makedirs(dst_dir, exist_ok=True)

    # Kopier og konverter filene
    for root, _, files in os.walk(temp_år_mappe):
        for file in files:
            src_file = os.path.join(root, file)
            rel_path = os.path.relpath(src_file, temp_år_mappe)
            dst_file = os.path.join(år_mappe, rel_path)

            if konverter_fil_til_geoparquet(src_file, dst_file):
                suksess_count += 1
            else:
                feil_count += 1

    return suksess_count, feil_count

def konverter_partisjonerte_filer_til_geoparquet(rotmappe):
    """
    Konverterer alle partisjonerte parquet-filer til GeoParquet format.
    """
    from shapely import wkb # NB!! Ikke fjern!
    feil_count = 0

    for root, _, files in os.walk(rotmappe):
        for file in files:
            if not file.endswith('.parquet'):
                continue

            parquet_path = os.path.join(root, file)
            try:
                # Les dataframe
                part_df = pd.read_parquet(parquet_path)

                # Hopp over hvis den ikke har geometry_wkb
                if 'geometry_wkb' not in part_df.columns:
                    continue

                # Konverter WKB tilbake til geometri
                part_df['geometry'] = part_df['geometry_wkb'].apply(lambda x: wkb.loads(x))
                part_df = part_df.drop(columns=['geometry_wkb'])

                # Lag GeoDataFrame
                part_gdf = gpd.GeoDataFrame(part_df, geometry='geometry', crs="EPSG:4326")

                # Skriv GeoParquet-filen
                part_gdf.to_parquet(parquet_path)
            except Exception as e:
                print(f"Feil ved konvertering av {parquet_path}: {e}")
                feil_count += 1

    if feil_count > 0:
        print(f"Advarsel: {feil_count} filer kunne ikke konverteres til GeoParquet")

def konverter_parquet_til_geoparquet(filsti, målfilsti, partisjon_kolonner=None):
    """
    Konverterer parquet-fil til GeoParquet-format med partisjonering.
    Args:
        filsti: Sti til parquet-filen
        målfilsti: Sti hvor GeoParquet-filen skal lagres
        partisjon_kolonner: Liste av kolonnenavn som skal brukes for partisjonering
    """
    try:
        df = pd.read_parquet(filsti)
    except Exception as e:
        print(f"Kunne ikke lese parquet-fil: {e}")
        return False

    # Håndter tidspartisjonering
    tidspartisjonering = False
    if partisjon_kolonner and 'date_time_utc' in partisjon_kolonner:
        partisjon_kolonner.remove('date_time_utc')
        tidspartisjonering = True

    # Legg til alle tidspartisjoneringskolonner, men vi vil bare partisjonere på time
    if tidspartisjonering:
        df = legg_til_tidspartisjoneringskolonner(df)
        # Sett opp partisjonering kun på 'hour'
        partisjon_kolonner = ['hour'] + (partisjon_kolonner or [])

    # Opprett GeoDataFrame
    gdf = opprett_geodataframe(df)
    if gdf is None:
        return False

    # Sjekk at alle partisjoneringskolonner finnes
    if partisjon_kolonner and not all(col in gdf.columns for col in partisjon_kolonner):
        print(f"Advarsel: Ikke alle partisjoneringskolonner finnes i datasettet")
        manglende = [col for col in partisjon_kolonner if col not in gdf.columns]
        print(f"Manglende kolonner: {manglende}")
        return False

    # Lagre med partisjonering (bare time)
    if partisjon_kolonner:
        return lagre_partisjonert_geoparquet(gdf, målfilsti, partisjon_kolonner)
    else:
        gdf.to_parquet(målfilsti)
        return målfilsti

def konverter_alle_parquet_filer(data_mappe, partisjon_kolonner=None):
    """
    Konverterer alle parquet-filer i en mappe til GeoParquet.
    """
    rå_mappe = os.path.join(data_mappe, "raw")
    prosessert_mappe = os.path.join(data_mappe, "processed")

    # Opprett mapper
    os.makedirs(rå_mappe, exist_ok=True)
    os.makedirs(prosessert_mappe, exist_ok=True)

    resultater = {
        "konvertert": [],
        "feilet": []
    }

    # Finn alle parquet-filer
    parquet_filer = [f for f in os.listdir(rå_mappe)
                     if f.lower().endswith('.parquet') and os.path.isfile(os.path.join(rå_mappe, f))]

    if not parquet_filer:
        print("Ingen parquet-filer funnet i råmappen")
        return resultater

    # Konverter hver fil
    for filnavn in parquet_filer:
        filsti = os.path.join(rå_mappe, filnavn)
        base_filnavn = os.path.splitext(filnavn)[0]
        målfilsti = os.path.join(prosessert_mappe, f"{base_filnavn}.parquet")

        # Konverter filen
        resultat = konverter_parquet_til_geoparquet(
            filsti,
            målfilsti,
            partisjon_kolonner.copy() if partisjon_kolonner else None,
        )

        if resultat:
            resultater["konvertert"].append(filsti)
        else:
            resultater["feilet"].append(filsti)
            print(f"Kunne ikke konvertere: {filsti}")

    return resultater

def vis_partisjoneringsstruktur(konvertert_sti):
    """
    Viser partisjoneringsstrukturen for en konvertert fil.
    """

    for root, dirs, files in os.walk(konvertert_sti, topdown=True, followlinks=False):
        nivå = root.replace(konvertert_sti, "").count(os.sep)
        innrykk = "    " * (nivå + 1)

        # Vis mappenavnet
        mappe_navn = os.path.basename(root)
        if mappe_navn:  # Ikke vis for rot-mappen
            print(f"{innrykk}- {mappe_navn}")

        # Vis antall filer i dypeste mapper
        if not dirs and files:
            print(f"{innrykk}  Inneholder {len(files)} filer")

def vis_datasett_info(filsti):
    """
    Viser informasjon om et GeoParquet datasett.
    """
    try:
        gdf = gpd.read_parquet(filsti)
        print(f"  Datasett størrelse: {len(gdf)} rader, {len(gdf.columns)} kolonner")
        return gdf
    except Exception as e:
        print(f"  Kunne ikke lese filen: {e}")
        return None

def start_konvertering(data_mappe="./data", partisjon_kolonner=None):
    """
    Start konverteringsprosessen fra Parquet til GeoParquet.
    Args:
        data_mappe: Sti til datamappen
        partisjon_kolonner: Liste av kolonner for partisjonering
    """
    start_tid = time.time()
    print(f"Starter konvertering og partisjonering av parquet-filer i {data_mappe} ...\n")
    print(f"Partisjonering vil skje på 'hour' og følgende kolonner: {partisjon_kolonner or []}")

    # Opprett og sjekk mappene
    mappestier = opprett_mappestruktur(data_mappe)
    rå_mappe = mappestier["rå_mappe"]

    # Utfør konverteringen
    resultater = konverter_alle_parquet_filer(data_mappe, partisjon_kolonner)

    print("Konvertering og partisjonering utført:")
    print(f"• Total behandlingstid: {time.time() - start_tid:.2f} sekunder")
    print(f"• Konverterte filer: {len(resultater['konvertert'])}")

    return resultater

In [19]:
mappestier = opprett_mappestruktur()

Mappestruktur opprettet.


In [27]:
# Kun konvertering
# resultater = start_konvertering(data_mappe="./data")

# Manuell konvertering og partisjonering
resultater = start_konvertering(data_mappe="./data", partisjon_kolonner=["date_time_utc", "ship_type"])

Starter konvertering og partisjonering av parquet-filer i ./data ...

Partisjonering vil skje på 'hour' og følgende kolonner: ['date_time_utc', 'ship_type']
Mappestruktur opprettet.
Konvertering og partisjonering utført:
• Total behandlingstid: 0.96 sekunder
• Konverterte filer: 2


In [22]:
import os
import datetime
from shapely.geometry import Point, Polygon

def definer_agder_polygon():
    """
    Definerer et forenklet polygon for Agder-kysten.
    Dette er et eksempel og bør erstattes med faktiske koordinater.

    Returns:
        Shapely Polygon som definerer Agder-kystlinjen
    """
    # Eksempelkoordinater for Agder-kysten (forenklet)
    agder_coords = [
        (6.8, 57.8),  # Sørvest
        (10.0, 57.8),  # Sørøst
        (10.0, 59.5),  # Nordøst
        (6.8, 59.5),  # Nordvest
        (6.8, 57.8)   # Lukk polygonet
    ]
    return Polygon(agder_coords)

def last_agder_geojson(geojson_sti):
    """
    Laster inn GeoJSON-fil med Agder fylkesgrense.

    Args:
        geojson_sti: Sti til GeoJSON-filen

    Returns:
        GeoDataFrame med Agder fylkesgrense
    """
    try:
        agder_gdf = gpd.read_file(geojson_sti)
        # Sikre at GeoDataFrame bruker WGS84 (EPSG:4326)
        if agder_gdf.crs != "EPSG:4326":
            agder_gdf = agder_gdf.to_crs("EPSG:4326")
        print(f"Agder fylkesgrense lastet inn fra {geojson_sti}")
        return agder_gdf
    except Exception as e:
        print(f"Kunne ikke laste GeoJSON-fil: {e}")
        return None

def filtrer_ais_data(
    rot_mappe,
    dato=None,  # datetime objekt for dato - påkrevd
    filtre=None,  # Dict med kolonner og verdier å filtrere på
    agder_polygon=None,  # GeoDataFrame eller Shapely polygon for Agder-kysten
    maks_skip=10  # Maksimalt antall unike skip å returnere
):
    """
    Filtrerer AIS-data basert på dato og geografisk område.

    Args:
        rot_mappe: Mappen hvor dataene er lagret
        dato: Dato for filtrering
        filtre: Dictionary med kolonnenavn og verdier for filtrering
        agder_polygon: Polygon som definerer Agder-kysten
        maks_skip: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    if not dato:
        print("Dato er påkrevd")
        return None

    if filtre is None:
        filtre = {}

    # Dato-komponenter for filtrering senere
    year, month, day = dato.year, dato.month, dato.day

    # Konverter dato til string for bruk i filnavn
    dato_str = dato.strftime('%Y-%m-%d')

    # Skriv ut parametere for debugging
    print(f"Filtrerer data for dato: {dato_str}")
    print(f"Søker i mappe: {rot_mappe}")

    # Sjekk om rot-mappen eksisterer
    if not os.path.exists(rot_mappe):
        print(f"Mappen {rot_mappe} finnes ikke")
        return None

    # Finn alle mapper i rot-mappen
    dato_mapper = []
    for element in os.listdir(rot_mappe):
        full_sti = os.path.join(rot_mappe, element)
        if os.path.isdir(full_sti):
            dato_mapper.append(element)

    if not dato_mapper:
        print(f"Ingen datamapper funnet i {rot_mappe}")
        return None

    print(f"Fant følgende datamapper: {dato_mapper}")

    # Finn alle parquet-filer rekursivt
    data_funnet = []

    for dato_mappe in dato_mapper:
        base_path = os.path.join(rot_mappe, dato_mappe)
        # Traverser mappestrukturen rekursivt
        for root, dirs, files in os.walk(base_path):
            # For time-partisjonering, sjekk fil-innhold istedenfor å basere på katalogstruktur
            for file in files:
                if file.endswith('.parquet'):
                    filsti = os.path.join(root, file)
                    data_funnet.append(filsti)

    if not data_funnet:
        print(f"Ingen parquet-filer funnet")
        return None

    print(f"Fant {len(data_funnet)} potensielle parquet-filer")

    # Les inn og filtrer dataene
    dfs = []
    for fil_sti in data_funnet:
        try:
            # Ekstraher partisjonsverdier fra stien (for time)
            time_value = None
            parts = fil_sti.split(os.sep)
            for part in parts:
                if part.startswith('hour='):
                    time_value = int(part.split('=')[1])

            # Les parquet-filen
            df = pd.read_parquet(fil_sti)

            # Legg til hour som kolonne hvis den ikke allerede finnes
            if time_value is not None and 'hour' not in df.columns:
                df['hour'] = time_value

            # Filtrer på dato (vi må gjøre dette manuelt siden vi kun partisjonerer på time)
            if 'year' in df.columns and 'month' in df.columns and 'day' in df.columns:
                df = df[(df['year'] == year) & (df['month'] == month) & (df['day'] == day)]

                if df.empty:
                    # Hopp over denne filen hvis ingen rader matcher datoen
                    continue

            # Utfør ekstra filtrering basert på filtre-parameteren
            skip_df = False
            for kolonne, verdi in filtre.items():
                if kolonne not in df.columns:
                    skip_df = True
                    break

                # For alle kolonner, filtrer på vanlig måte
                if isinstance(verdi, list):
                    if not df[kolonne].isin(verdi).any():
                        skip_df = True
                        break
                else:
                    if not (df[kolonne] == verdi).any():
                        skip_df = True
                        break

            if not skip_df:
                # Gjør ytterligere filtrering på dataframe-nivå
                for kolonne, verdi in filtre.items():
                    if kolonne in df.columns:
                        if isinstance(verdi, list):
                            df = df[df[kolonne].isin(verdi)]
                        else:
                            df = df[df[kolonne] == verdi]

                if not df.empty:
                    print(f"  - Beholder {len(df)} rader fra {fil_sti}")
                    dfs.append(df)

        except Exception as e:
            print(f"Feil ved lesing av {fil_sti}: {str(e)}")

    if not dfs:
        print("Ingen data lest inn")
        return None

    # Slå sammen alle dataframes
    combined_df = pd.concat(dfs, ignore_index=True)
    print(f"Kombinert dataframe har {len(combined_df)} rader")

    # Konverter til GeoDataFrame hvis koordinater finnes
    if 'longitude' in combined_df.columns and 'latitude' in combined_df.columns:
        # Opprett geometrikolonne fra longitude og latitude
        geometry = [Point(xy) for xy in zip(combined_df['longitude'], combined_df['latitude'])]
        geo_df = gpd.GeoDataFrame(combined_df, geometry=geometry, crs="EPSG:4326")

        # Filtrer basert på Agder-polygon hvis gitt
        if agder_polygon is not None:
            print("Filtrerer basert på Agder-polygon")
            # Sjekk om agder_polygon er et GeoDataFrame eller et Shapely-objekt
            if isinstance(agder_polygon, gpd.GeoDataFrame):
                # Sørg for at CRS er likt
                if agder_polygon.crs != geo_df.crs:
                    agder_polygon = agder_polygon.to_crs(geo_df.crs)

                # Spatial join - dette kan ta litt tid for store datasett
                geo_df = gpd.sjoin(geo_df, agder_polygon, how="inner", predicate="within")
            else:
                # Antar at agder_polygon er et Shapely-objekt
                geo_df = geo_df[geo_df.geometry.within(agder_polygon)]

            print(f"Etter geografisk filtrering: {len(geo_df)} rader")

        # Sorter etter nyeste data først for hvert skip
        if 'timestamp' in geo_df.columns:
            geo_df = geo_df.sort_values(by='timestamp', ascending=False)

        # Begrens til unike skip
        if 'mmsi' in geo_df.columns or 'ship_name' in geo_df.columns:
            id_kolonne = 'mmsi' if 'mmsi' in geo_df.columns else 'ship_name'
            unike_skip = geo_df[id_kolonne].unique()
            print(f"Fant {len(unike_skip)} unike skip")

            if len(unike_skip) > maks_skip:
                print(f"Begrenser til {maks_skip} skip")
                unike_skip = unike_skip[:maks_skip]

            geo_df = geo_df[geo_df[id_kolonne].isin(unike_skip)]

            # Hent første rad for hvert unikt skip for å få en kompakt liste
            geo_df = geo_df.drop_duplicates(subset=[id_kolonne])

        return geo_df
    else:
        print("Mangler koordinater (longitude/latitude)")
        return None

def filtrer_skip_agder(rot_mappe, dato, antall_skip=10):
    """
    Bruker et utvidet polygon for Agder-kysten for å filtrere skipene.

    Args:
        rot_mappe: Mappen hvor de konverterte dataene er lagret
        dato: Datoen å filtrere på (datetime objekt)
        antall_skip: Maksimalt antall skip å returnere

    Returns:
        GeoDataFrame med filtrerte skip
    """
    # Bruk utvidet polygon
    agder_polygon = definer_agder_polygon()
    print("Bruker utvidet polygon for Agder-kysten")

    # Finn skip innenfor det utvidede området
    skip_innenfor_agder = filtrer_ais_data(
        rot_mappe=rot_mappe,
        dato=dato,
        filtre=None,
        agder_polygon=agder_polygon,
        maks_skip=antall_skip
    )

    if skip_innenfor_agder is None or skip_innenfor_agder.empty:
        print(f"Ingen skip funnet innenfor Agder-område den {dato.strftime('%d.%m.%Y')}")
        return None

    print(f"Fant {len(skip_innenfor_agder)} skip innenfor Agder-område på datoen {dato.strftime('%d.%m.%Y')}")
    return skip_innenfor_agder

In [24]:
# Filtrer skip i Agder for en bestemt dato
rot_mappe = "data/processed"
dato = datetime.datetime(2024, 12, 1)  # Velg dato
skip_agder = filtrer_skip_agder(rot_mappe, dato)

# Vis skipene på tabellform
if skip_agder is not None and not skip_agder.empty:
    # Velg kolonner å vise
    kolonner_å_vise = ['mmsi', 'ship_name', 'ship_type', 'longitude', 'latitude']

    # Velg bare kolonner som faktisk finnes i datasettet
    tilgjengelige_kolonner = [kol for kol in kolonner_å_vise if kol in skip_agder.columns]

    styled_tabell = skip_agder[tilgjengelige_kolonner].style\
        .format({'longitude': '{:.4f}', 'latitude': '{:.4f}'})\
        .set_caption('Skip innenfor Agder-området')\
        .set_table_styles([
            {'selector': 'th', 'props': [('background-color', '#f2f2f2'),
                                        ('color', '#333'),
                                        ('font-weight', 'bold')]},
            {'selector': 'td', 'props': [('padding', '5px')]}
        ])

    display(styled_tabell)
else:
    print("Ingen skip å vise.")

Bruker utvidet polygon for Agder-kysten
Filtrerer data for dato: 2024-12-01
Søker i mappe: data/processed
Fant følgende datamapper: ['hais_2024-12-01.snappy.parquet']
Fant 24 potensielle parquet-filer
  - Beholder 793 rader fra data/processed/hais_2024-12-01.snappy.parquet/hour=19/ship_type=51/40a4cd95889b4789a913dfb423a832c6-0.parquet
  - Beholder 796 rader fra data/processed/hais_2024-12-01.snappy.parquet/hour=21/ship_type=51/40a4cd95889b4789a913dfb423a832c6-0.parquet
  - Beholder 797 rader fra data/processed/hais_2024-12-01.snappy.parquet/hour=17/ship_type=51/40a4cd95889b4789a913dfb423a832c6-0.parquet
  - Beholder 796 rader fra data/processed/hais_2024-12-01.snappy.parquet/hour=10/ship_type=51/40a4cd95889b4789a913dfb423a832c6-0.parquet
  - Beholder 799 rader fra data/processed/hais_2024-12-01.snappy.parquet/hour=5/ship_type=51/40a4cd95889b4789a913dfb423a832c6-0.parquet
  - Beholder 804 rader fra data/processed/hais_2024-12-01.snappy.parquet/hour=2/ship_type=51/40a4cd95889b4789a913df

,mmsi,ship_name,longitude,latitude
0,257959900,RESCUE UTVAER,8.3833,58.2450
1,258012180,SAR NICOLAI JARLSBY,8.0333,58.1467
2,258258500,RESCUE STORMBULL,8.7783,58.4617
4,257004200,RC LILLESAND,8.3850,58.2464
382,257895900,RS127 TBN,8.3847,58.2466
399,258002830,RS ERLING SKJALGSSON,6.8087,58.0925
793,257565700,RESCUE 157 BILL,7.4769,58.0203


In [25]:
import folium
from folium import plugins
import pandas as pd
import geopandas as gpd

def visualiser_skip(skip_gdf, agder_polygon=None, tittel="Skip innenfor Agder"):
    """
    Visualiserer skip på et interaktivt kart med Folium.

    Args:
        skip_gdf: GeoDataFrame med skip
        agder_polygon: GeoDataFrame eller Shapely polygon med Agder-området (valgfri)
        tittel: Tittel på kartet

    Returns:
        Folium Map objekt
    """
    if skip_gdf is None or skip_gdf.empty:
        print("Ingen skip å visualisere")
        return None

    # Beregn senterpunkt for kartet basert på skipene
    map_center = [skip_gdf.geometry.y.mean(), skip_gdf.geometry.x.mean()]

    # Opprett kartet
    m = folium.Map(
        location=map_center,
        zoom_start=12,
        tiles='OpenStreetMap',
        control_scale=True
    )

    # Legg til Agder-polygon hvis tilgjengelig
    if agder_polygon is not None:
        if isinstance(agder_polygon, gpd.GeoDataFrame):
            # Hvis det er en GeoDataFrame, konverter til GeoJSON og legg til
            folium.GeoJson(
                data=agder_polygon,
                name="Agder fylke",
                style_function=lambda x: {
                    'fillColor': '#aaaaff',
                    'color': '#0000ff',
                    'weight': 2,
                    'fillOpacity': 0.1
                }
            ).add_to(m)
        else:
            # Hvis det er et Shapely polygon, konverter koordinater og legg til
            if isinstance(agder_polygon, Polygon):
                coords = list(agder_polygon.exterior.coords)
                # Konverter til format [lat, lon] som Folium forventer
                folium_coords = [[y, x] for x, y in coords]

                folium.Polygon(
                    locations=folium_coords,
                    color='blue',
                    weight=2,
                    fill=True,
                    fill_color='#aaaaff',
                    fill_opacity=0.1,
                    name='Agder fylke'
                ).add_to(m)

    # Opprett en MarkerCluster for skipene
    marker_cluster = plugins.MarkerCluster(name="Skip").add_to(m)

    # Legg til markører for hvert skip
    for idx, row in skip_gdf.iterrows():
        # Samle informasjon for popup
        popup_info = "<table>"
        for col in skip_gdf.columns:
            if col != 'geometry' and pd.notna(row[col]):
                popup_info += f"<tr><th>{col}</th><td>{row[col]}</td></tr>"
        popup_info += "</table>"

        # Opprett popup
        popup = folium.Popup(popup_info, max_width=300)

        # Hent skipnavn
        skip_navn = row.get('ship_name', f"Skip {idx}")

        # Opprett ikon
        icon = folium.Icon(icon='ship', prefix='fa', color='blue')

        # Legg til markør
        folium.Marker(
            location=[row.geometry.y, row.geometry.x],
            popup=popup,
            tooltip=skip_navn,
            icon=icon
        ).add_to(marker_cluster)

    # Legg til fullskjermsknapp
    plugins.Fullscreen().add_to(m)

    # Legg til lagkontroll (for å slå av/på lag)
    folium.LayerControl().add_to(m)

    # Legg til tegneforklaring
    legend_html = """
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
                background-color: white; padding: 10px; border-radius: 5px;
                border: 2px solid grey; font-size: 14px;">
      <p><b>Tegnforklaring</b></p>
      <p>
        <i class="fa fa-ship fa-1x" style="color:blue"></i> Skip<br>
        <span style="background-color: #aaaaff; opacity: 0.1; border: 1px solid blue;">&nbsp;&nbsp;&nbsp;&nbsp;</span> Agder fylke
      </p>
    </div>
    """
    m.get_root().html.add_child(folium.Element(legend_html))

    #m.fit_bounds(m.get_bounds())

    return m

def finn_skip_rundt_punkt(skip_gdf, punkt, buffer_avstand=500):
    """
    Filtrerer skip som er innenfor en gitt avstand fra et punkt.

    Args:
        skip_gdf: GeoDataFrame med skip
        punkt: Tuple (longitude, latitude)
        buffer_avstand: Avstand i meter

    Returns:
        GeoDataFrame med filtrerte skip
    """
    if skip_gdf is None or skip_gdf.empty:
        return None

    try:
        from shapely.geometry import Point
        import pyproj
        from shapely.ops import transform
        from functools import partial

        # Opprett punkt fra koordinater
        senterpunkt = Point(punkt)

        # Projiser til et lokalt koordinatsystem som bruker meter som enheter (UTM-sone)
        # Finn UTM-sone basert på longitude
        utm_sone = int((punkt[0] + 180) / 6) + 1
        utm_crs = f"EPSG:326{utm_sone}"  # Nordlig halvkule

        # Transformer funksjonen for å lage buffer i meter
        project_to_utm = pyproj.Transformer.from_crs(
            "EPSG:4326", utm_crs, always_xy=True).transform
        project_to_wgs84 = pyproj.Transformer.from_crs(
            utm_crs, "EPSG:4326", always_xy=True).transform

        # Projiser punkt til UTM, lag buffer i meter, projiser tilbake til WGS84
        utm_punkt = transform(project_to_utm, senterpunkt)
        buffer_utm = utm_punkt.buffer(buffer_avstand)
        buffer_wgs84 = transform(project_to_wgs84, buffer_utm)

        # Filtrer for skip innenfor bufferen
        skip_innenfor_buffer = skip_gdf[skip_gdf.geometry.within(buffer_wgs84)]
        print(f"Fant {len(skip_innenfor_buffer)} skip innenfor {buffer_avstand}m fra punktet")

        return skip_innenfor_buffer, buffer_wgs84

    except Exception as e:
        print(f"Feil ved buffering: {str(e)}")
        return None, None

def visualiser_skip_rundt_punkt(skip_gdf, punkt, buffer_avstand=500, agder_polygon=None):
    """
    Visualiserer skip innenfor en buffer rundt et gitt punkt.

    Args:
        skip_gdf: GeoDataFrame med skip
        punkt: Tuple (longitude, latitude)
        buffer_avstand: Avstand i meter
        agder_polygon: GeoDataFrame eller Shapely polygon med Agder-området (valgfri)

    Returns:
        Folium Map objekt
    """
    # Finn skip innenfor buffer
    skip_innenfor_buffer, buffer_wgs84 = finn_skip_rundt_punkt(skip_gdf, punkt, buffer_avstand)

    if skip_innenfor_buffer is None or skip_innenfor_buffer.empty:
        print(f"Ingen skip funnet innenfor {buffer_avstand}m fra punktet {punkt}")

        # Opprett et kart selv om ingen skip ble funnet, for å vise punktet og bufferen
        map_center = [punkt[1], punkt[0]]  # [lat, lon]
        m = folium.Map(
            location=map_center,
            zoom_start=14,
            tiles='CartoDB positron',
            control_scale=True
        )

        # Legg til flere bakgrunnskart
        folium.TileLayer('OpenStreetMap').add_to(m)
        folium.TileLayer('CartoDB dark_matter').add_to(m)
        folium.TileLayer('Stamen Terrain').add_to(m)

        # Legg til markør for senterpunkt
        folium.Marker(
            location=[punkt[1], punkt[0]],
            popup=f"Koordinater: {punkt}",
            tooltip="Senterpunkt",
            icon=folium.Icon(icon='crosshairs', prefix='fa', color='red')
        ).add_to(m)

        # Legg til sirkel for buffer
        folium.Circle(
            location=[punkt[1], punkt[0]],
            radius=buffer_avstand,
            color='red',
            fill=True,
            fill_color='#ff7777',
            fill_opacity=0.2,
            popup=f"Buffer: {buffer_avstand}m"
        ).add_to(m)

        # Legg til fullskjermsknapp og lagkontroll
        plugins.Fullscreen().add_to(m)
        folium.LayerControl().add_to(m)

        return m

    # Opprett kart
    tittel = f"Skip innenfor {buffer_avstand}m fra punktet"
    m = visualiser_skip(skip_innenfor_buffer, agder_polygon, tittel)

    # Legg til markør for senterpunkt
    folium.Marker(
        location=[punkt[1], punkt[0]],
        popup=f"Koordinater: {punkt}",
        tooltip="Senterpunkt",
        icon=folium.Icon(icon='crosshairs', prefix='fa', color='red')
    ).add_to(m)

    # Legg til sirkel for buffer
    folium.Circle(
        location=[punkt[1], punkt[0]],
        radius=buffer_avstand,
        color='red',
        fill=True,
        fill_color='#ff7777',
        fill_opacity=0.2,
        popup=f"Buffer: {buffer_avstand}m"
    ).add_to(m)

    return m

In [26]:
# Visualiser skipene
kart = visualiser_skip(skip_agder, definer_agder_polygon())

# Vis kartet i notebooken
kart

# Visualiser skip rundt et bestemt punkt
punkt = (8.0, 58.15)  # (longitude, latitude)
kart_punkt = visualiser_skip_rundt_punkt(skip_agder, punkt, 6000)

# Vis kartet
kart_punkt

Fant 1 skip innenfor 6000m fra punktet
